In [1]:
import pandas as pd

In [2]:
diamonds =  pd.read_csv('train.csv')

In [3]:
diamonds.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0.53,Very Good,G,SI1,63.4,54.0,5.09,5.13,3.24,7.057
1,1,0.41,Ideal,D,SI1,63.0,56.0,4.80,4.75,3.01,6.824
2,2,0.32,Ideal,I,VS2,61.6,56.0,4.37,4.39,2.70,6.107
3,3,0.31,Ideal,H,VVS2,61.2,56.0,4.34,4.37,2.66,6.390
4,4,1.35,Premium,J,VS2,60.5,56.0,7.19,7.12,4.33,8.741


In [4]:
pd.get_dummies(diamonds, columns=["cut", "color", "clarity"])

,id,carat,depth,table,x,y,z,price,cut_Fair,cut_Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,0.53,63.4,54.0,5.09,5.13,3.24,7.057,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0.41,63.0,56.0,4.80,4.75,3.01,6.824,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,0.32,61.6,56.0,4.37,4.39,2.70,6.107,0,0,...,1,0,0,0,0,0,0,1,0,0
3,3,0.31,61.2,56.0,4.34,4.37,2.66,6.390,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,1.35,60.5,56.0,7.19,7.12,4.33,8.741,0,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,0.52,61.2,58.0,5.16,5.20,3.17,7.508,0,0,...,0,0,0,0,0,0,0,1,0,0
40451,40451,0.52,62.0,55.0,5.14,5.17,3.19,7.232,0,0,...,0,0,0,0,1,0,0,0,0,0
40452,40452,0.73,63.5,58.0,5.68,5.72,3.62,8.065,0,0,...,0,0,0,0,0,0,0,1,0,0
40453,40453,0.31,56.9,59.0,4.45,4.48,2.54,6.629,1,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
diamonds = pd.get_dummies(diamonds, columns=["cut", "color", "clarity"], drop_first=True)

In [6]:
from sklearn.model_selection import train_test_split
target = "price"

In [7]:
X = diamonds.drop(target, axis=1)
y = diamonds[target]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.2)

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
model = GradientBoostingRegressor(n_estimators=3000)

In [12]:
params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
          'max_depth': [4, 6],
          'min_samples_leaf': [3, 5, 9, 17],
          'max_features': [1, 0.3, 0.1]}
grid_search = GridSearchCV(model,
                           param_grid=params,
                           cv=2,
                           n_jobs=3,
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 10.6min
[Parallel(n_jobs=3)]: Done 192 out of 192 | elapsed: 42.7min finished


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(n_estimators=3000),
             n_jobs=3,
             param_grid={'learning_rate': [0.1, 0.05, 0.02, 0.01],
                         'max_depth': [4, 6], 'max_features': [1, 0.3, 0.1],
                         'min_samples_leaf': [3, 5, 9, 17]},
             verbose=1)

In [13]:
grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 4,
 'max_features': 0.3,
 'min_samples_leaf': 3}

In [14]:
param = grid_search.best_params_

In [17]:
model = GradientBoostingRegressor(**param)

In [18]:
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=4, max_features=0.3, min_samples_leaf=3)

In [21]:
y_pred_test = model.predict(X_test)
y_pred_test

array([6.27614679, 8.46066745, 6.75159955, ..., 9.09890222, 9.4599691 ,
       8.11456312])

In [22]:
y_pred_train = model.predict(X_train)
y_pred_train

array([8.61978592, 9.66991027, 8.12627356, ..., 8.49234452, 9.62154251,
       6.78796448])

In [23]:
from sklearn.metrics import mean_squared_error
mean_squared_error(
    y_train,
    y_pred_train
)

0.01703480395828671

In [24]:
mean_squared_error(
    y_test,
    y_pred_test
)

0.018380309216536828

## TEST

In [25]:
diamonds_test =  pd.read_csv('test.csv')

In [26]:
diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.33,Ideal,H,IF,61.9,55.0,4.44,4.42,2.74
1,1,0.41,Ideal,E,VS2,61.8,54.0,4.79,4.76,2.95
2,2,0.91,Very Good,E,SI2,62.5,59.0,6.16,6.23,3.87
3,3,0.42,Very Good,G,VS2,62.6,57.0,4.76,4.80,2.99
4,4,0.54,Ideal,G,IF,61.5,56.0,5.28,5.25,3.24


In [27]:
pd.get_dummies(diamonds_test, columns=["cut", "color", "clarity"])

,id,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0,0.33,61.9,55.0,4.44,4.42,2.74,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1,0.41,61.8,54.0,4.79,4.76,2.95,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,2,0.91,62.5,59.0,6.16,6.23,3.87,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3,0.42,62.6,57.0,4.76,4.80,2.99,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,0.54,61.5,56.0,5.28,5.25,3.24,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.55,61.7,56.4,5.26,5.30,3.25,0,0,1,...,0,0,0,0,1,0,0,0,0,0
13481,13481,1.12,60.6,59.0,6.77,6.70,4.08,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13482,13482,0.37,61.5,57.0,4.63,4.60,2.84,0,0,1,...,0,0,0,0,1,0,0,0,0,0
13483,13483,0.54,59.9,63.0,5.25,5.30,3.16,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [28]:
diamonds_test = pd.get_dummies(diamonds_test, columns=["cut", "color", "clarity"], drop_first=True)

In [30]:
X_test_f = diamonds_test

In [31]:
model.predict(X_test_f)

array([6.73839626, 6.92591605, 8.24495998, ..., 6.62163427, 7.33161329,
       8.13208223])

In [32]:
diamonds_test['price']= model.predict(X_test_f)

In [33]:
prediction_3 = diamonds_test[['id', 'price']]

In [34]:
prediction_3.set_index('id', inplace= True)

In [35]:
prediction_3.to_csv("data/prediction_GradientBoosting.csv")